In [41]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

import os
import warnings
warnings.filterwarnings('ignore')


In [42]:
raw_path = os.path.join('..','data','raw')
input_path = os.path.join('..','data','model_input')

In [43]:
books = pd.read_csv(os.path.join(raw_path,'Books.csv'))
ratings = pd.read_csv(os.path.join(raw_path,'Ratings.csv'))
users = pd.read_csv(os.path.join(raw_path,'Users.csv'))

In [44]:
book_ratings = books.merge(ratings ,on = 'ISBN')
user_rating = users.merge(ratings , on = 'User-ID')

In [45]:
book_ratings.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,0


In [46]:
book_ratings.groupby('Book-Title')['Book-Rating'].count().reset_index().rename(columns = {'Book-Rating':'Num-Ratings' })

,Book-Title,Num-Ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [47]:
book_num_ratings = book_ratings.groupby('Book-Title')['Book-Rating'].count().reset_index().rename(columns = {'Book-Rating':'Num-Ratings' })
book_avg_ratings = book_ratings.groupby('Book-Title')['Book-Rating'].mean().reset_index().rename(columns = {'Book-Rating':'Avg-Ratings' })
final_rating = book_num_ratings.merge(book_avg_ratings , on = 'Book-Title').merge(books[['Book-Title','Book-Author','Image-URL-M']].groupby('Book-Title').first(), on = 'Book-Title')

In [48]:
books.to_pickle(os.path.join(input_path,'books.pkl'))

## Recomendação Baseada em Popularidade

Recomendação dos livros mais avaliados e com nota mais alta

In [49]:
popular_books = final_rating[final_rating['Num-Ratings'] > 250].sort_values(by = 'Avg-Ratings'  , ascending= False).reset_index(drop = True).head(50)

In [50]:
popular_books.head(15)

,Book-Title,Num-Ratings,Avg-Ratings,Book-Author,Image-URL-M
0,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804,J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...
1,Harry Potter and the Goblet of Fire (Book 4),387,5.824289,J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...
2,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410,J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...
3,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...
4,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453,J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...
5,The Hobbit : The Enchanting Prelude to The Lor...,281,5.007117,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339681.0...
6,The Fellowship of the Ring (The Lord of the Ri...,368,4.948370,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339703.0...
7,Harry Potter and the Sorcerer's Stone (Harry P...,575,4.895652,J. K. Rowling,http://images.amazon.com/images/P/059035342X.0...
8,"The Two Towers (The Lord of the Rings, Part 2)",260,4.880769,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339711.0...
9,To Kill a Mockingbird,510,4.700000,Harper Lee,http://images.amazon.com/images/P/0446310786.0...


In [51]:
popular_books.to_pickle(os.path.join(input_path,'popular_books.pkl'))

## Recomendação por Filtragem Colaborativa
Recomendação baseada na similaridade entre usuários:
* Apenas os usuários que deram mais de 200 notas serão levados em conta
* Apenas livros com pelo menos 50 notas

In [52]:
x = book_ratings.groupby('User-ID').count()['Book-Rating'] > 200
educated_users  = x[x].index

book_ratings = book_ratings[book_ratings['User-ID'].isin(educated_users)]

y  = book_ratings.groupby('Book-Title')['Book-Rating'].count() >= 50
famous_books = y[y].index

final = book_ratings[book_ratings['Book-Title'].isin(famous_books)]

In [53]:
x[x].index

Index([   254,   2276,   2766,   2977,   3363,   4017,   4385,   6251,   6323,
         6543,
       ...
       271705, 273979, 274004, 274061, 274301, 274308, 275970, 277427, 277639,
       278418],
      dtype='int64', name='User-ID', length=811)

In [54]:
pt = final.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating').fillna(0)

In [55]:
pt.head()

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
pt.to_pickle(os.path.join(input_path,'pt.pkl'))

In [57]:
similarity_scores = cosine_similarity(pt)

def recommend(book_name):
    index = np.where(pt.index == book_name)[0][0]
    similar_books = sorted(enumerate(similarity_scores[index]),key= lambda x: x[1], reverse =True)[1:6]
    
    for i in similar_books:
        print(pt.index[i[0]])

In [58]:
recommend("4 Blondes")

The House of the Spirits
Pride and Prejudice
Pleading Guilty
Seabiscuit
Notes from a Small Island


In [60]:
pd.DataFrame(similarity_scores).to_pickle(os.path.join(input_path,'similarity_scores.pkl'))